In [1]:
# coding: utf-8
# In[1]:
### For reproducibility, you have set random.seed before load keras.
### Tensorflow are a little faster, but could not reproduce results with Keras 2.0
# In[2]:

# -*- coding: utf-8 -*-
from __future__ import print_function
from __future__ import division

In [2]:
from ipywidgets import Dropdown
from IPython.display import display
import os

In [3]:
datasets = { 'ag_news': ('18wYKf0IKTu2fWUUZQ_DPonZFfh-RkUkU','ag_news_csv.zip', 'zip'),
             'amazon_review_full': ('13quYvDyutGnWSSwhYg4CopAaj4OXxRrR','amazon_review_full_csv.zip', 'zip'),
             'amazon_review_polarity': ('1DGJITPcVJYBD13KaA26wl6xhfYQ2Kfol','amazon_review_polarity_csv.zip', 'zip'),
             'dbpedia': ('1zRBM3-CCms0GqVqmeuoHA4GaeFYGr2KK','dbpedia_csv.zip', 'zip'),
             'sogou_news': ('1ghut0YEGDqHzIpevXCLx4Yi3dqHoZ9lk','sogou_news_csv.zip', 'zip'),
             'yahoo_answers': ('1Yxp4Y_3BgK2qX2sT9e7VuSGJYTiMfpLM','yahoo_answers_csv.zip', 'zip'),
             'yelp_review_full': ('1YDwclFFYy7vMKHPQzUJjyojp4VGht-xL','yelp_review_full_csv.zip', 'zip'),
             'yelp_review_polarity': ('1P3N6EHdZUp1ggpRAJCt6Bd7oDDIkM9l5','yelp_review_polarity_csv.zip', 'zip')
           }

    
#https://drive.google.com/open?id=13quYvDyutGnWSSwhYg4CopAaj4OXxRrR
#https://drive.google.com/open?id=1DGJITPcVJYBD13KaA26wl6xhfYQ2Kfol
#https://drive.google.com/open?id=1zRBM3-CCms0GqVqmeuoHA4GaeFYGr2KK
#https://drive.google.com/open?id=1ghut0YEGDqHzIpevXCLx4Yi3dqHoZ9lk
#https://drive.google.com/open?id=1Yxp4Y_3BgK2qX2sT9e7VuSGJYTiMfpLM
#https://drive.google.com/open?id=1YDwclFFYy7vMKHPQzUJjyojp4VGht-xL
#https://drive.google.com/open?id=1P3N6EHdZUp1ggpRAJCt6Bd7oDDIkM9l5

In [4]:
wid = Dropdown(options=list(datasets), description='Select dataset to use')
display(wid)

In [5]:
# Preparing data and downloading if necessary

In [6]:
data_folder = 'data' + os.sep

In [7]:
if not os.path.exists(data_folder):
    os.mkdir(data_folder)

In [8]:
def download_file_from_google_drive(id, destination):
    import requests
    
    def get_confirm_token(response):
        for key, value in response.cookies.items():
            if key.startswith('download_warning'):
                return value

        return None

    def save_response_content(response, destination):
        CHUNK_SIZE = 32768

        with open(destination, "wb") as f:
            for chunk in response.iter_content(CHUNK_SIZE):
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)

    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    




In [9]:
dbase = wid.value

In [10]:
print('The selected dataset is', dbase)

The selected dataset is ag_news


In [11]:
if not os.path.exists(data_folder + dbase+'_csv'):
    #os.mkdir(data_folder + selected_dataset)
    id, file_name, file_type = datasets[dbase]
    download_file_from_google_drive(id, data_folder +os.sep+file_name)
    
    if file_type == 'tgz':
        import tarfile
        tar = tarfile.open(data_folder + os.sep+file_name)
        tar.extractall(data_folder + os.sep)
        
    if file_type =='zip':
        import zipfile
        zip_ref = zipfile.ZipFile(data_folder +os.sep+file_name, 'r')
        zip_ref.extractall(data_folder + os.sep)
        zip_ref.close()
        
    ## if file exists, delete it ##
    if os.path.isfile(data_folder +os.sep+file_name):
        os.remove(data_folder +os.sep+file_name) 

In [12]:
import sys
import os

os.environ['KERAS_BACKEND']='theano'

import numpy as np
np.random.seed(7)  

In [13]:
# set parameters:
if not os.path.exists('params'):
    os.mkdir('params')


path_test = 'data/'+dbase+'_csv/test.csv'
path_train = 'data/'+dbase+'_csv/train.csv'

epochs=10
batch_size = 32
batch_gen=10000
filter_sizes = [3,3,3,3]

best_acc=0

cod_type='0'

len_words=128
len_cod_word=256

In [14]:
#Whether to save model parameters

model_output = 'params/Huffman_'+dbase+'_output.csv'
model_prediction_output = 'params/Huffman_'+dbase
model_real='params/Huffman_'+dbase+'_real'

model_keras_model='params/Huffman_'+dbase+'_model.json'
model_keras_wheights='params/Huffman_'+dbase+'_weights.h5'


In [15]:
import json
import datetime
import string
import re
import math
import random
import time
import io


In [16]:
'''def shuffle_file(path_train, path_train_shuffled):
    from os import listdir
    from os.path import isfile, join

    #onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
    exist_file =isfile(path_train_shuffled)

    doc=[]
    if not exist_file:
        with io.open(path_train, encoding='utf-8') as f:
            for line in f:
                doc.append(line)

        p = np.random.permutation(len(doc))

        with open(path_train_shuffled,'w', encoding='utf-8') as f:
            for idp in p:
                f.write (doc[p[idp]])
            f.close() 
    


return (not exist_file)'''

"def shuffle_file(path_train, path_train_shuffled):\n    from os import listdir\n    from os.path import isfile, join\n\n    #onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]\n    exist_file =isfile(path_train_shuffled)\n\n    doc=[]\n    if not exist_file:\n        with io.open(path_train, encoding='utf-8') as f:\n            for line in f:\n                doc.append(line)\n\n        p = np.random.permutation(len(doc))\n\n        with open(path_train_shuffled,'w', encoding='utf-8') as f:\n            for idp in p:\n                f.write (doc[p[idp]])\n            f.close() \n    \n\n\nreturn (not exist_file)"

In [17]:
path_train_shuffled = 'data/'+dbase+'_csv/train_shuffled.csv'

'''# verifying if already exists a shuffle training data file

created_shuffled = shuffle_file(path_train, path_train_shuffled)

# reseting random seed
np.random.seed(7) 
if created_shuffled:
    print('shuffled file generated ')'''

"# verifying if already exists a shuffle training data file\n\ncreated_shuffled = shuffle_file(path_train, path_train_shuffled)\n\n# reseting random seed\nnp.random.seed(7) \nif created_shuffled:\n    print('shuffled file generated ')"

In [18]:
# generating alphabet ordered by frequency

from collections import Counter

with io.open(path_train_shuffled, encoding='utf-8') as f:
    c = Counter()
    for line in f:
        text=line.lower()
        c += Counter(list(text))

alpha=sorted(c.items(), key=lambda c: c[1],reverse=True)
# ranked alphabet by frequency
letter = [item[0] for item in alpha]
print('Alphabet: ',len(letter))

Alphabet:  57


In [19]:
def shuffle_matrix(a, b):
    #assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [20]:
def show_time(start):
    end=time.time()
    m, s = divmod(end-start, 60)
    h, m = divmod(m, 60)
    return ("%d:%02d:%02d" % (h, m, s))

In [21]:
def clean_str(text):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
       
    #Removing consecutive spaces, lowercase words and removing whitespaces from both sides
    
    text=re.sub(' +', ' ', text)   
    text = text.lower()
    text = text.strip()
    
    return text

In [22]:
know_words={}

In [23]:
cod = {}
decod={}
for i, l in enumerate(letter):
    cod[l]="1"+cod_type*i+"1"
    decod["1"+cod_type*i+"1"]=l


In [24]:
def compress(text,know_words=know_words):
    if text in know_words:
        huff=know_words[text]
    else:
        missing= "1"+cod_type*(len(letter)+1)+"1"
        huff =''
        for l in text:
            huff += cod.get(l,missing)
        know_words[text]=huff
    return huff

In [25]:
def encode_text(text, len_words, len_cod_word,reverse=False):
    # Iterate over the loaded data and create a matrix of size len_cod_word x len_word
    # Longer are cutted, shorter are padded
    # Chars not in the vocab are encoded to a missing value.

    input_data = np.zeros((len_words,len_cod_word))

    # eliminating some chars
    sent_words = clean_str(text).split() 
    
   
    if reverse:
        # limiting to len_words size - treating last words as more important
        sent_words = sent_words[-np.min([len_words, len(sent_words)]):]
    else:
        sent_words = sent_words[:len_words]   
        
    sent_array = np.zeros((len_words,len_cod_word))

    counter = 0
    for j, w in enumerate(sent_words):
        cod_w = compress(w)
              
        # limmiting cod_w to len_cod_word
        xl=len(cod_w)
        cod_w = cod_w[:np.min([len_cod_word, len(cod_w)])]
        
        for i, c in enumerate(cod_w):
            sent_array[j,i] = int(c)

    input_data= sent_array

    return input_data


In [26]:
### category number, # validating docs, # testing docs

with io.open(path_train_shuffled, encoding='utf-8') as f:
    cat_output=0
    ndocs_t=0
    for line in f:
        
        ndocs_t+=1
        
        data_yx=line.split(',')
            
        cat = data_yx[0]
        cat=re.sub('"', '', cat)
        cat=int(cat)
        
        if cat>cat_output:
            cat_output=cat

with io.open(path_test, encoding='utf-8') as f:
    ndocs_v=0
    for line in f:
        ndocs_v+=1

print('Trainning Documents: ',ndocs_t)      
print('Validating Documents: ',ndocs_v)  
print('Categories: ',cat_output)

Trainning Documents:  120000
Validating Documents:  7600
Categories:  4


In [27]:
x_train=[]
y_train=[]
x_evaluate=[]
y_evaluate=[]

with io.open(path_train_shuffled, encoding='utf-8') as f:
    for line in f:
        
        data_yx=line.split(',')
        cat = data_yx[0]
        cat=re.sub('"', '', cat)

        result=np.zeros((cat_output))
        result[np.array(int(cat))-1]=1
            
        y_train.append(result)
            
        txt=""
        for col_txt in data_yx[1:]:
            txt = txt+" "+col_txt 
            
        x_train.append(txt)
        
with io.open(path_test, encoding='utf-8') as f:
    for line in f:
        
        data_yx=line.split(',')
        cat = data_yx[0]
        cat=re.sub('"', '', cat)

        result=np.zeros((cat_output))
        result[np.array(int(cat))-1]=1
            
        y_evaluate.append(result)
            
        txt=""
        for col_txt in data_yx[1:]:
            txt = txt+" "+col_txt 
            
        x_evaluate.append(txt)

In [28]:
def gen_sample_from_memory(x,y,cat_output,nb_sample,len_words,len_cod_word,shuffle=True):
    
    p = np.random.permutation(len(x))
    p = p[:nb_sample]
    
    n_sample =len(p)
    sent_data_x = np.zeros((n_sample, len_words, len_cod_word))
    sent_data_y = np.zeros((n_sample, cat_output))
    
    sample =0
    for c_sample in p:
        sent_data_y[sample,:] = y[c_sample]
        sent_data_x[sample,:,:] = encode_text(x[c_sample],len_words,len_cod_word)
            
        sample+=1
                
    if shuffle:
        sent_data_x,sent_data_y = shuffle_matrix(sent_data_x,sent_data_y)
                
    return (sent_data_x,sent_data_y)


In [29]:
def gen_continuous_from_memory(x,y,cat_output,batch_gen,len_words,len_cod_word,shuffle=True):
   
    for n_start in range(0,len(x),batch_gen):  
            
        n_sample =np.min((len(x)-n_start,batch_gen))
        sent_data_x = np.zeros((n_sample, len_words, len_cod_word))
        sent_data_y = np.zeros((n_sample, cat_output))

        for sample in range(0,n_sample):

            sent_data_y[sample,:] = y[n_start+sample]
            sent_data_x[sample,:,:] = encode_text(x[n_start+sample],len_words,len_cod_word)

        if shuffle:
            sent_data_x,sent_data_y = shuffle_matrix(sent_data_x,sent_data_y)            
        
        yield (sent_data_x,sent_data_y)

In [30]:
def gen_sample_from_file(path,cat_output,nb_sample,len_words,len_cod_word,shuffle=True):
    
    with io.open(path, encoding='utf-8') as f:
        ndocs=0
        for line in f:
            ndocs+=1
    
    p = np.random.permutation(ndocs)
    p = p[:nb_sample]
    
    
    n_sample =len(p)
    sent_data_x = np.zeros((n_sample, len_words, len_cod_word))
    sent_data_y = np.zeros((n_sample, cat_output))
    
    
    with io.open(path, encoding='utf-8') as f:
        
        sample = 0
        nline =0
        while sample<len(p):

            line = f.readline()
            
            if nline in p:

                data_yx=line.split(',')
                cat = data_yx[0]
                cat=re.sub('"', '', cat)

                result=np.zeros((cat_output))
                result[np.array(int(cat))-1]=1
            
                sent_data_y[sample,:] = result
            
                txt=""
                for col_txt in data_yx[1:]:
                    txt = txt+" "+col_txt 
                               
                sent_data_x[sample,:,:] = encode_text(txt,len_words,len_cod_word)
            
                sample+=1
            
            nline+=1
            
            
        if shuffle:
            sent_data_x,sent_data_y = shuffle_matrix(sent_data_x,sent_data_y)
                
    return (sent_data_x,sent_data_y)


In [31]:
def gen_continuous_from_file(path,cat_output,batch_gen,len_words,len_cod_word,shuffle=True):
    
    from os.path import isfile, join
    
    with io.open(path, encoding='utf-8') as f:
        ndocs=0
        for line in f:
            ndocs+=1
    
    with io.open(path, encoding='utf-8') as f:
        for n_start in range(0,ndocs,batch_gen):  
            
            n_sample =np.min((ndocs-n_start,batch_gen))
            sent_data_x = np.zeros((n_sample, len_words, len_cod_word))
            sent_data_y = np.zeros((n_sample, cat_output))

            for sample in range(0,n_sample):

                line = f.readline()
                
                data_yx=line.split(',')
                cat = data_yx[0]
                cat=re.sub('"', '', cat)

                result=np.zeros((cat_output))
                result[np.array(int(cat))-1]=1

                sent_data_y[sample,:] = result

                txt=""
                for col_txt in data_yx[1:]:
                    txt = txt+" "+col_txt 

                sent_data_x[sample,:,:] = encode_text(txt,len_words,len_cod_word)

            if shuffle:
                sent_data_x,sent_data_y = shuffle_matrix(sent_data_x,sent_data_y)            
        
            yield (sent_data_x,sent_data_y)

In [32]:

# -*- coding: utf-8 -*-
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Flatten, Dropout
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D, Embedding, Concatenate, Dropout
from keras.optimizers import RMSprop
from keras.models import model_from_json


#Define what the input shape looks like

inputs = Input(shape=(len_words, len_cod_word,),  dtype='float32')
l_lstm = Dropout(.1)(LSTM(300)(inputs))
l_dense = Dense(128, activation='sigmoid')(l_lstm)
pred = Dense(cat_output, activation='softmax')(l_dense)
 
model = Model(inputs=inputs, outputs=pred)

# Reloading Saved params
#model.load_weights("crepe_model_weights.h5", by_name=True)
#print ('weights loaded...')
    

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

Using Theano backend.
Using cuDNN version 5110 on context None
Preallocating 9011/11264 Mb (0.800000) on cuda0
Mapped name None to device cuda0: GeForce GTX 1080 Ti (0000:01:00.0)


In [33]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 256)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 300)               668400    
_________________________________________________________________
dropout_1 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               38528     
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 516       
Total params: 707,444
Trainable params: 707,444
Non-trainable params: 0
_________________________________________________________________


In [34]:
from keras.callbacks import CSVLogger
logger = CSVLogger(model_output, separator=',', append=True)

# random sample of at max 10000 texts, just for reference while training

#xv_ref,yv_ref=gen_sample_from_file(path_test,cat_output,1000,len_words,len_cod_word,shuffle=False)
xv_ref,yv_ref=gen_sample_from_memory(x_evaluate,y_evaluate,cat_output,1000,len_words,len_cod_word,shuffle=False)


In [35]:
#def evaluating(epoch,path_test,cat_output,batch_gen,len_words,len_cod_word)
def evaluating(epoch, x_evaluate,y_evaluate,cat_output,batch_gen,len_words,len_cod_word):

    #ev_batch = gen_continuous_from_file(path_test,cat_output,batch_gen,len_words,len_cod_word)
    ev_batch = gen_continuous_from_memory(x_evaluate,y_evaluate,cat_output,batch_gen,len_words,len_cod_word)

    pred=list()
    real=list()
    for x_ev, y_ev in ev_batch:
        prediction = model.predict(x_ev)
        pred=pred+list(prediction)
        real = real +list(y_ev)
        
    r=np.argmax(real,axis=1)
    p=np.argmax(pred,axis=1)
    
    acc = np.sum(r==p)/len(r)
    
    np.savetxt(model_real+'_epoch_'+str(e)+'.txt', real,fmt='%1i')
    np.savetxt(model_prediction_output+'_epoch_'+str(e)+'_acc_'+'{:06.4f}'.format(acc)+'.txt', pred,fmt='%06.4f')
    
    return acc

In [36]:

total_start = time.time()

for e in range(0,epochs):
    
    epoch_start = time.time()
    
    trained= 0
    print ()
    print ('epoch:',e)
    print ()
    
    #gen=gen_continuous_from_file(path_train_shuffled,cat_output,batch_gen,len_words,len_cod_word)
    gen=gen_continuous_from_memory(x_train,y_train,cat_output,batch_gen,len_words,len_cod_word)

    for x,y in gen:
        
        batch_start = time.time()
        
        model.fit(x,y,batch_size,epochs=1,verbose=1,validation_data=(xv_ref,yv_ref),callbacks=[logger])
        
        batch_end = time.time()
        
        trained+=len(x)
        print('Epoch '+str(e)+'  Trained: {:6.2f}%'.format(100*trained/ndocs_t))
        print('Time Elapsed:  Total: '+show_time(total_start)+' Epoch:'+show_time(epoch_start)+ ' Batch: '+show_time(batch_start) )
        
        
        print ()
    
    #evaluating 
    acc= evaluating(e,x_evaluate,y_evaluate,cat_output,batch_gen,len_words,len_cod_word)

    
    print ('Epoch '+str(e)+': {:06.4f}'.format(acc))
    print('Time Elapsed:  Total: '+show_time(total_start)+' Epoch:'+show_time(epoch_start))
    
    if acc>best_acc:
        best_acc=acc

        # serialize model to JSON
        with open(model_keras_model, "w") as json_file:
            json_file.write(model.to_json())
            
        # serialize best weights to HDF5
        model.save_weights(model_keras_wheights)
        print("Saved model to disk")
    
    


epoch: 0

Train on 10000 samples, validate on 1000 samples
Epoch 1/1
10000/10000 [==============================] - 26s - loss: 1.3924 - acc: 0.2550 - val_loss: 1.3953 - val_acc: 0.2390
Epoch 0  Trained:   8.33%
Time Elapsed:  Total: 0:01:28 Epoch:0:01:28 Batch: 0:01:17

Train on 10000 samples, validate on 1000 samples
Epoch 1/1
10000/10000 [==============================] - 25s - loss: 1.3924 - acc: 0.2470 - val_loss: 1.3847 - val_acc: 0.2610
Epoch 0  Trained:  16.67%
Time Elapsed:  Total: 0:02:04 Epoch:0:02:04 Batch: 0:00:25

Train on 10000 samples, validate on 1000 samples
Epoch 1/1
10000/10000 [==============================] - 25s - loss: 1.3890 - acc: 0.2500 - val_loss: 1.3907 - val_acc: 0.2500
Epoch 0  Trained:  25.00%
Time Elapsed:  Total: 0:02:41 Epoch:0:02:41 Batch: 0:00:25

Train on 10000 samples, validate on 1000 samples
Epoch 1/1
10000/10000 [==============================] - 25s - loss: 1.3899 - acc: 0.2543 - val_loss: 1.3971 - val_acc: 0.2540
Epoch 0  Trained:  33.33%
T

KeyboardInterrupt: 